In [38]:
import os
import geopandas as gpd
import pandas as pd
import fiona
from shapely.geometry import Polygon, MultiPolygon

# === CONFIG ===
catchment_files = [
    "Datasets/na/hybas_lake_na_lev08_v1c.shp"
]
lake_file = "Datasets/lakes/CCILakesV202.shp"
lake_id_column = "Lake_ID"
output_file = "Datasets/final/lake_catchments_na_cleaned.shp"

# === STEP 1: Load and clean HydroBASINS catchments ===
def clean_geometries(gdf):
    cleaned = []
    for geom in gdf.geometry:
        if geom is None:
            cleaned.append(None)
        elif geom.geom_type in ["Polygon", "MultiPolygon"]:
            cleaned.append(geom)
        elif geom.geom_type == "GeometryCollection":
            polys = [g for g in geom.geoms if isinstance(g, (Polygon, MultiPolygon))]
            cleaned.append(polys[0] if polys else None)
        else:
            cleaned.append(None)
    gdf["geometry"] = cleaned
    return gdf[~gdf["geometry"].isnull()]

catchments_cleaned = []
for path in catchment_files:
    raw = gpd.read_file(path)
    cleaned = clean_geometries(raw)
    catchments_cleaned.append(cleaned)

catchments_gdf = gpd.GeoDataFrame(pd.concat(catchments_cleaned, ignore_index=True), crs=cleaned.crs)

# === STEP 2: Load lakes and spatially clip to HydroBASINS area ===
lakes_gdf = gpd.read_file(lake_file).to_crs(catchments_gdf.crs)
lakes_clipped = gpd.clip(lakes_gdf, catchments_gdf)

# === STEP 3: Assign lakes to catchments ===
def assign_lakes_to_catchments(catchments_gdf, lakes_gdf, lake_id_column):
    lakes_gdf = lakes_gdf.to_crs(catchments_gdf.crs)
    intersecting = gpd.sjoin(catchments_gdf, lakes_gdf[[lake_id_column, 'geometry']], how='inner', predicate='intersects')
    intersecting['match_method'] = 'intersect'
    matched_ids = set(intersecting[lake_id_column])
    unmatched = lakes_gdf[~lakes_gdf[lake_id_column].isin(matched_ids)].copy()
    print(f"ℹ️ {len(unmatched)} lakes didn’t intersect any catchment. Assigning nearest catchment...")

    if not unmatched.empty:
        projected_crs = "EPSG:3395"
        unmatched = unmatched.to_crs(projected_crs)
        catchments_proj = catchments_gdf.to_crs(projected_crs)
        unmatched["geometry"] = unmatched.geometry.centroid

        nearest = gpd.sjoin_nearest(
            unmatched[[lake_id_column, 'geometry']],
            catchments_proj,
            how='left',
            distance_col="dist_to_catchment"
        )
        nearest['match_method'] = 'nearest'
        nearest = nearest.to_crs(catchments_gdf.crs)
        result = pd.concat([intersecting, nearest], ignore_index=True)
    else:
        result = intersecting

    return result, unmatched

lake_catchments, unmatched_lakes = assign_lakes_to_catchments(catchments_gdf, lakes_clipped, lake_id_column)

# === STEP 4: Clean up final lake_catchments
lake_catchments[lake_id_column] = lake_catchments[lake_id_column].astype(float)

# Restore lake polygon geometries (in case any are centroids from nearest)
lake_polygons = lakes_clipped.set_index(lake_id_column)[["geometry"]]
lake_catchments["geometry"] = lake_catchments[lake_id_column].map(lake_polygons["geometry"])

# Remove any duplicate join columns or repeat rows
lake_catchments = lake_catchments.loc[:, ~lake_catchments.columns.duplicated()]
lake_catchments = lake_catchments.drop(columns=[c for c in lake_catchments.columns if c.lower().startswith("index_") or c.lower().startswith("dist_")], errors="ignore")
lake_catchments = lake_catchments.drop_duplicates(subset=["Lake_ID", "HYBAS_ID", "geometry"])

# Final geometry validation (only polygons)
lake_catchments = clean_geometries(lake_catchments)

# === STEP 5: Save to shapefile using Fiona (with large Lake_ID float) ===
fiona.supported_drivers['ESRI Shapefile'] = 'rw'
lake_catchments = lake_catchments.rename(columns=lambda x: x[:10])
schema = gpd.io.file.infer_schema(lake_catchments)
schema['properties']['Lake_ID'] = 'float:18.0'

os.makedirs(os.path.dirname(output_file), exist_ok=True)
lake_catchments.to_file(output_file, driver="ESRI Shapefile", engine="fiona", schema=schema)

print(f"✅ Final cleaned shapefile saved: {output_file}")
print(f"🎯 Unique Lake_IDs: {lake_catchments['Lake_ID'].nunique()}")
print(f"❗ Unmatched lakes that used nearest: {len(unmatched_lakes)}")


ℹ️ 0 lakes didn’t intersect any catchment. Assigning nearest catchment...
✅ Final cleaned shapefile saved: Datasets/final/lake_catchments_na_cleaned.shp
🎯 Unique Lake_IDs: 466
❗ Unmatched lakes that used nearest: 0


In [42]:
expected_lakes = lakes_clipped["Lake_ID"].nunique()
print(f"🎯 Expected number of lakes in HydroBASINS area: {expected_lakes}")
assigned_lakes = lake_catchments["Lake_ID"].nunique()
print(f"✅ Lake_IDs assigned to catchments: {assigned_lakes}")
if expected_lakes == assigned_lakes:
    print("✅ All lakes in the HydroBASINS area have been assigned to catchments.")
else:
    print(f"❌ Mismatch: {expected_lakes - assigned_lakes} lakes are missing from the final output.")


🎯 Expected number of lakes in HydroBASINS area: 466
✅ Lake_IDs assigned to catchments: 466
✅ All lakes in the HydroBASINS area have been assigned to catchments.
